In [44]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

# Nos conectamos a la base de datos

In [63]:
load_dotenv()
secret = os.getenv("client_secret")


In [6]:
client = MongoClient("mongodb://localhost:27017/")
db = client.companies

In [7]:
companies = db.companies.find()

In [9]:
data_companies = pd.DataFrame(companies)

In [62]:
data_companies.head(3)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN


In [11]:
data_companies.shape

(18801, 42)

In [12]:
data_companies["category_code"].value_counts()

web                 3787
software            2736
games_video         1083
mobile              1018
other                986
advertising          928
enterprise           742
ecommerce            688
consulting           637
network_hosting      626
public_relations     533
search               394
biotech              373
hardware             368
cleantech            305
semiconductor        167
security             156
analytics             66
social                49
finance               49
news                  48
education             36
music                 33
messaging             30
travel                25
medical               25
legal                 25
photo_video           23
health                23
manufacturing         19
sports                13
fashion               10
real_estate           10
automotive             9
hospitality            8
transportation         7
nanotech               5
nonprofit              4
design                 4
local                  1


In [13]:
acquisition = db.companies.find({"$and":[
                               {"founded_year": {"$gte": 2000}},
                               {"total_money_raised":{"$regex":'M'}},
                               {"$or": [
                                   {"category_code": "web" } ,
                                   {"category_code": "software" } ,
                                   {"category_code": "games_video" } ,
                                   ]} ,                                ]
                               },
                                {"_id": 0, "crunchbase_url": 0, "products": 0,
                                "acquisition": 0, "acquisitions": 0, "video_embeds": 0,
                                "screenshots": 0, "external_links": 0, "partners": 0,
                                "image": 0, "blog_feed_url": 0, "blog_url":0, "deadpooled_day":0,
                                "deadpooled_year":0, "deadpooled_month":0, "deadpooled_url":0, "ipo":0,
                                "permalink":0, "homepage_url":0, "tag_list":0, "email_addres": 0, "providerships": 0,
                                "milestones": 0, "twitter_username": 0, "updated_at": 0, "email_address":0, "investments":0,
                               }
                                    )

In [14]:
acquisition = pd.DataFrame(acquisition)

In [15]:
acquisition.shape

(1036, 16)

In [17]:
acquisition = acquisition.replace(np.nan, 0)

In [18]:
acquisition.isnull().sum()

name                   0
category_code          0
number_of_employees    0
founded_year           0
founded_month          0
founded_day            0
alias_list             0
phone_number           0
description            0
created_at             0
overview               0
relationships          0
competitions           0
total_money_raised     0
funding_rounds         0
offices                0
dtype: int64

In [19]:
latitude = db.companies.find({"offices.latitude":1})
longitude = db.companies.find({"offices.longitude":1})


In [20]:
def geopoint(x):
    for ele in x:
        longitude = ele["longitude"]
        latitude = ele["latitude"]
        
        return{"type": "Point", "coordinates": {'longitude': longitude , 'latitude': latitude}}

acquisition["geo"] = acquisition["offices"].apply(geopoint)

In [21]:
b=acquisition[acquisition["geo"]=={'type': 'Point', 'coordinates': {'longitude': None, 'latitude': None}}].index
acquisition.drop(b,axis=0,inplace=True)

In [22]:
acquisition.dropna(subset=['geo'], inplace=True)

In [23]:
acquisition.geo.isnull().sum()

0

In [25]:
acquisition.to_json("office.json",orient="records", lines=True)

In [26]:
acquisition['geo'].value_counts()

{'type': 'Point', 'coordinates': {'longitude': -122.419204, 'latitude': 37.775196}}      12
{'type': 'Point', 'coordinates': {'longitude': -95.712891, 'latitude': 37.09024}}         8
{'type': 'Point', 'coordinates': {'longitude': 34.7595, 'latitude': 32.0554}}             4
{'type': 'Point', 'coordinates': {'longitude': -122.159928, 'latitude': 37.437328}}       3
{'type': 'Point', 'coordinates': {'longitude': -73.986951, 'latitude': 40.756054}}        3
                                                                                         ..
{'type': 'Point', 'coordinates': {'longitude': -105.228284, 'latitude': 40.026422}}       1
{'type': 'Point', 'coordinates': {'longitude': 6.921546, 'latitude': 50.945289}}          1
{'type': 'Point', 'coordinates': {'longitude': -118.462876, 'latitude': 34.054318}}       1
{'type': 'Point', 'coordinates': {'longitude': -122.1797053, 'latitude': 37.4520239}}     1
{'type': 'Point', 'coordinates': {'longitude': -122.2103414, 'latitude': 37.4207

In [27]:
db = client.office

In [28]:
near = db.office.find()
df = pd.DataFrame(near)

In [29]:
def geonear(geopoint, maxdistance=1000):
    return db.office.find({
        "geo":{
            "$near":{
                "$geometry":geopoint,
                "$maxDistance":maxdistance
            }}})

San_Francisco = {'type': 'Point', 'coordinates': [-122.393229, 37.781265]}

In [30]:
empresas = pd.DataFrame(geonear(San_Francisco,1000))

def empresas_cercanas(local):
    names = []
    lats =[]
    long=[]
    for i in empresas['geo']:
        long.append(i['coordinates']['longitude'])
        lats.append(i['coordinates']['latitude'])
    for el in empresas['category_code']:
        names.append(el)
    local = {"name":names, "latitude":lats, "longitude":long}
    return pd.DataFrame(local)
empresas_cercanas = empresas_cercanas(empresas)
empresas_cercanas.head(2)

,name,latitude,longitude
0,games_video,37.781265,-122.393229
1,games_video,37.780716,-122.393913


In [50]:
import requests
import json

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
 client_id='HIX31VQGRL1GODLF1XYQO5ERLH1WBFWLLCAPWYHLNATD55JC',
 client_secret= secret,
 v='20180323',
 ll='37.781265, -122.393229',
 query='starbucks',
 radius="1000"
 #limit=5
)

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)


In [32]:
import folium
from folium.plugins import HeatMap
from folium.plugins import MeasureControl

In [33]:
def starbucks(lugar):
   names=[]
   lats=[]
   long=[]
   for e in data['response']['groups'][0]['items']:
       names.append(e['venue']['name'])
       lats.append(e['venue']['location']['lat'])
       long.append(e['venue']['location']['lng'])
   lugar = {"name":names, "latitude":lats, "longitude":long}
   return pd.DataFrame(lugar)
starbucks = starbucks(data)
starbucks.head(2)

,name,latitude,longitude
0,Starbucks,37.778704,-122.396727
1,Starbucks,37.789651,-122.396622


In [68]:
url = 'https://api.foursquare.com/v2/venues/explore'

params_1 = dict(
  client_id='HIX31VQGRL1GODLF1XYQO5ERLH1WBFWLLCAPWYHLNATD55JC',
  client_secret= '3CVWFRJLEFHRKVT3HCMGEAK1ZOW1NZU3OAUOSRP2AGOTLIC2',
  v='20180323',
  ll='37.781265, -122.393229',
  query='4bf58dd8d48988d11f941735',
  radius = 1000

)
resp = requests.get(url=url, params=params_1)
data = json.loads(resp.text)




In [60]:
def club(lugar):
   names=[]
   lats=[]
   long=[]
   for e in data['response']['groups'][0]['items']:
       names.append(e['venue']['name'])
       lats.append(e['venue']['location']['lat'])
       long.append(e['venue']['location']['lng'])
   lugar = {"name":names, "latitude":lats, "longitude":long}
   return pd.DataFrame(lugar)
club = club(data)
club.head()

,name,latitude,longitude
0,H.D. Buttercup,37.777490,-122.394582
1,AMC Metreon 16,37.784654,-122.403189
2,Tazza D'Amore,37.776636,-122.394897


In [67]:
url = 'https://api.foursquare.com/v2/venues/explore'

parames = dict(
  client_id='HIX31VQGRL1GODLF1XYQO5ERLH1WBFWLLCAPWYHLNATD55JC',
  client_secret= '3CVWFRJLEFHRKVT3HCMGEAK1ZOW1NZU3OAUOSRP2AGOTLIC2',
  v='20180323',
  ll='37.781265, -122.393229',
  categoryId = '5744ccdfe4b0c0459246b4c7',
  radius = 1000

)
respoco = requests.get(url=url, params=parames)
dataco = json.loads(respoco.text)


In [65]:
def guarde(lugare):
   name=[]
   lat=[]
   lng=[]
   for e in dataco['response']['groups'][0]['items']:
       name.append(e['venue']['name'])
       lat.append(e['venue']['location']['lat'])
       lng.append(e['venue']['location']['lng'])
   
   lugare = {"name":name, "latitude":lat, "longitude":lng}
   return pd.DataFrame(lugare)
guarde = guarde(dataco)
guarde.head()

,name,latitude,longitude
0,Bright Horizons at 2nd Street,37.784808,-122.394899
1,Chirp Connecting Families and Nannies LLC,37.785786,-122.392112
2,Kids by the Bay - Financial District,37.785548,-122.398033
3,Bark Avenue,37.775338,-122.394974
4,Modern Education Family Childcare IV,37.773490,-122.391494


In [66]:
map_starbucks = folium.Map(location=[37.781265, -122.393229], width=750, height=500, zoom_start=15)

for index, row in starbucks.iterrows():
    folium.Marker([row['latitude'], row['longitude']],
                        radius=2,
                        icon=folium.Icon(color='red', icon='cloud'), # Icono nube, hay más en la documentación
                        
                       ).add_to(map_starbucks)
   
for index, row in guarde.iterrows():
    folium.Marker([row['latitude'], row['longitude']],
                        radius=2,
                        icon=folium.Icon(color='blue', icon='info'), 
                        
                       ).add_to(map_starbucks)
    folium.Circle(location=[37.781265, -122.393229],
                    radius=50
                   ).add_to(map_starbucks)
    
    folium.Circle(location=[37.781265, -122.393229],
                    radius=1000
                   ).add_to(map_starbucks)

for index, row in club.iterrows(): 
    folium.Marker([row['latitude'], row['longitude']],
                        radius=2,
                        icon=folium.Icon(icon='glass'), 
                        fill_color="#F35C50",
                       ).add_to(map_starbucks)
    
for index, row in empresas_cercanas.iterrows(): 
    folium.Marker([row['latitude'], row['longitude']],
                        radius=2,
                        icon=folium.Icon(color= 'green', icon='cloud' ),  
                        fill_color="#F35C50"
                       ).add_to(map_starbucks)

map_starbucks.add_child(MeasureControl())